Referencing https://www.analyticsvidhya.com/blog/2016/06/quick-guide-build-recommendation-engine-python/


In [48]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import time
from datetime import datetime
import os as os

In [49]:
os.getwd()

AttributeError: module 'os' has no attribute 'getwd'

In [8]:
#read in required fields of user details
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv("C:\\Users\\gftitman\\Desktop\\RecommenderSystems\\ml-100k\\ml-100k\\u.user", sep='|', names=u_cols,
 encoding='latin-1')

In [9]:
users.head(5)

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [10]:
#read in the required columns for movie ratings dataframes 
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv("C:\\Users\\gftitman\\Desktop\\RecommenderSystems\\ml-100k\\ml-100k\\u.data", sep='\t', names=r_cols,
 encoding='latin-1')

In [11]:
ratings.head(5)

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [12]:
#reading in required columns from items dataframe for movie details

i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv("C:\\Users\\gftitman\\Desktop\\RecommenderSystems\\ml-100k\\ml-100k\\u.item", sep='|', names=i_cols,
 encoding='latin-1')

In [13]:
items.head(5)

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


#### basic model using just the ratings dataset 

group the movies and get a count of the ratings

In [14]:
ratings_grouped = ratings.groupby(['movie_id'],as_index=False)[['rating']].agg(['mean','count'])
ratings_count_20 = ratings_grouped[ratings_grouped['rating']['count'] >=20]
ratings_count_20.loc[ratings_count_20['rating']['mean'].idxmax()]

rating  mean       4.491071
        count    112.000000
Name: 408, dtype: float64

so here we see that people have voted for movie id 408 to be the most popular with 112 votes

#### Aside - Issue with the method used by Analytics Vidhya in this link : 

The popularity based method used in the article mentions movies with movie id 1500 and 1293 as popular movies. Let's explore these movies. 

In [16]:
ratings.loc[ratings['movie_id'] == 1500 ]

,user_id,movie_id,rating,unix_timestamp
6126,279,1500,5,875306613
95364,781,1500,5,879634096


Only two people have rated this movie! Given the small sample size, can we generalize and say that a majority of the population will like it and recommend? NO! 

Likewise, the next popular movie also has only three people who have rated it. 

In [17]:
ratings.loc[ratings['movie_id'] == 1293 ]

,user_id,movie_id,rating,unix_timestamp
23044,489,1293,5,891446623
45350,519,1293,5,883250148
49046,146,1293,5,891458080


Which is why, I created a cut off threshold of 20, i.e, consider the average ratings of those movies that have been rated by at least 20 people. 

#### Now let's build a popularity engine using a classifier model 


In [20]:
ratings_train = pd.read_csv('C:\\Users\\gftitman\\Desktop\\RecommenderSystems\\ml-100k\\ml-100k\\ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('C:\\Users\\gftitman\\Desktop\\RecommenderSystems\\ml-100k\\ml-100k\\ua.test', sep='\t', names=r_cols, encoding='latin-1')

In [21]:
ratings_train.head(5)

,user_id,movie_id,rating,unix_timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [22]:
merge_1 = pd.merge(ratings_train,users,left_on=['user_id'],right_on=['user_id'])
merge_2 = pd.merge(merge_1,items,left_on=['movie_id'],right_on=['movie id'])
train = merge_2[['movie_id','rating','unknown', 'Action', 'Adventure', 'Animation', 'Children\'s',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western','age','sex','occupation']]


In [23]:
merge_test_1 = pd.merge(ratings_test,users,left_on=['user_id'],right_on=['user_id'])
merge_test_2 = pd.merge(merge_test_1,items,left_on=['movie_id'],right_on=['movie id'])
test = merge_test_2[['movie_id','rating','unknown', 'Action', 'Adventure', 'Animation', 'Children\'s',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western','age','sex','occupation']]

In [24]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [25]:
neigh = KNeighborsClassifier(n_neighbors=3)
y = train[['rating']]

In [26]:
train.drop(['rating'],inplace=True,axis=1)

In [27]:
neigh.fit(train,y)

C:\Users\gftitman\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [28]:
y_test = test[['rating']]

In [29]:
test.drop(['rating'],axis=1,inplace=True)

In [30]:
test.head(5)

,movie_id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
0,20,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20,0,0,0,0,0,0,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
test.columns

Index(['movie_id', 'unknown', 'Action', 'Adventure', 'Animation', 'Children's',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western', 'age', 'sex_F', 'sex_M', 'occupation_administrator',
       'occupation_artist', 'occupation_doctor', 'occupation_educator',
       'occupation_engineer', 'occupation_entertainment',
       'occupation_executive', 'occupation_healthcare', 'occupation_homemaker',
       'occupation_lawyer', 'occupation_librarian', 'occupation_marketing',
       'occupation_none', 'occupation_other', 'occupation_programmer',
       'occupation_retired', 'occupation_salesman', 'occupation_scientist',
       'occupation_student', 'occupation_technician', 'occupation_writer'],
      dtype='object')

In [32]:
#copy and paste the above into a list 

col_list = ['movie_id','unknown', 'Action', 'Adventure', 'Animation', 'Children\'s',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western', 'age', 'sex_F', 'sex_M', 'occupation_administrator',
       'occupation_artist', 'occupation_doctor', 'occupation_educator',
       'occupation_engineer', 'occupation_entertainment',
       'occupation_executive', 'occupation_healthcare', 'occupation_homemaker',
       'occupation_lawyer', 'occupation_librarian', 'occupation_marketing',
       'occupation_none', 'occupation_other', 'occupation_programmer',
       'occupation_retired', 'occupation_salesman', 'occupation_scientist',
       'occupation_student', 'occupation_technician', 'occupation_writer']
movie_list = list(items['movie id'])

We are creating a dataframe from one test subject with all movie_id and then using the predict() function to
get ratings of that user for each of the movies. 

In [33]:
x = []
for j in movie_list:
    x.append([])
    for i in range(len(col_list)):
        x[j-1].append(test[col_list[i]].ix[1])

In [34]:
#df is the test dataframe

df = pd.DataFrame(x,columns=col_list)

df['movie_id'] = pd.Series(movie_list)

In [35]:
predict_ratings = neigh.predict(df)

In [36]:
movie = max(predict_ratings)

In [37]:
recommend_movie = []
for i in range(len(predict_ratings)):
    if predict_ratings[i] == movie:
        recommend_movie.append(i)

recommend_movie is a list of movies that the user is predicted to give the highest ratings for. 
But, is this good enough? Can we recommend all of these movies? In what order do we recommend these movies? Well, the solution is simple and naive. Calculate the "age" of the movie in the recommend_movie list and recommend to the user top 10 most recently released movies! 

In [38]:
release_date = items[['movie id','release date','movie title']]

In [39]:
recommend_df = pd.DataFrame({'movie_id' : recommend_movie})

In [40]:
merged_df = pd.merge(release_date,recommend_df,left_on=['movie id'],right_on = ['movie_id'],how='right')

In [41]:
def date_time(s):
    return datetime.strptime(s,'%d-%b-%Y')

merged_df['date_time']=merged_df['release date'].apply(date_time)


In [42]:
def difference(s):
    diff = pd.Timedelta(pd.datetime.now().date() - s.date())
    return diff.days

merged_df['time_delta'] = merged_df['date_time'].apply(difference)

In [43]:
merged_df.head(5)

,movie id,release date,movie title,movie_id,date_time,time_delta
0,7,01-Jan-1995,Twelve Monkeys (1995),7,1995-01-01,8152
1,8,01-Jan-1995,Babe (1995),8,1995-01-01,8152
2,13,30-Oct-1995,Mighty Aphrodite (1995),13,1995-10-30,7850
3,21,16-Feb-1996,Muppet Treasure Island (1996),21,1996-02-16,7741
4,22,16-Feb-1996,Braveheart (1995),22,1996-02-16,7741


In [44]:
top_recommendations = merged_df.sort(['time_delta'],ascending=True)

C:\Users\gftitman\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [45]:
#These are the 10 movies that are recommended to the user. 

top_recommendations.head(10)

,movie id,release date,movie title,movie_id,date_time,time_delta
119,1191,01-Feb-1998,"Letter From Death Row, A (1998)",1191,1998-02-01,7025
52,301,19-Sep-1997,In & Out (1997),301,1997-09-19,7160
51,299,22-Aug-1997,Hoodlum (1997),299,1997-08-22,7188
136,1589,23-May-1997,Schizopolis (1996),1589,1997-05-23,7279
118,1190,04-Apr-1997,That Old Feeling (1997),1190,1997-04-04,7328
137,1598,14-Mar-1997,City of Industry (1997),1598,1997-03-14,7349
135,1588,21-Feb-1997,Salut cousin! (1996),1588,1997-02-21,7370
114,1142,14-Feb-1997,When We Were Kings (1996),1142,1997-02-14,7377
18,100,14-Feb-1997,Fargo (1996),100,1997-02-14,7377
121,1280,01-Jan-1997,Gang Related (1997),1280,1997-01-01,7421


#### Fin. 